<a href="https://colab.research.google.com/github/slazur83/Tableau/blob/main/Weight_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import pandas as pd
import glob
import os
import json
import numpy as np
from datetime import datetime
from google.colab import userdata
from google.colab import files

In [51]:
config_file_path = "/content/drive/MyDrive/Colab Notebooks/config.json"

with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

google_sheets_moja_waga = config['google_sheets_moja_waga']

**Google Sheets**

In [52]:
document_key = google_sheets_moja_waga.split('/')[-2]
google_sheets_moja_waga = f'https://docs.google.com/spreadsheets/d/{document_key}/export?format=csv&gid=0'
df = pd.read_csv(google_sheets_moja_waga, header=1)

In [53]:
df = df.iloc[1:, 1:]
df.columns = ['Date', 'Time', 'Weight']
df.replace('-', np.nan, inplace=True)
df['Weight'] = df['Weight'].str.replace(',', '.').astype(float)
df_google_sheets = df

In [54]:
duplicated_rows = df.duplicated()
print(f"Number of duplicated rows: {duplicated_rows.sum()}")
if duplicated_rows.sum() > 0:
  df = df.drop_duplicates()
  print (f"Found {duplicated_rows.sum()} rows. Deleted.")

Number of duplicated rows: 0


In [55]:
print(f'Number of entries: {len(df_google_sheets)}')
print(f'Date of first entry: {df_google_sheets["Date"].iloc[0]}')
print(f'Date of last entry: {df_google_sheets["Date"].iloc[-1]}')

Number of entries: 50
Date of first entry: 2020-02-02
Date of last entry: 2020-11-29


**Garmin Connect**



In [56]:
garmin_connect_masa_ciala = config['garmin_connect_masa_ciala']
file_paths = glob.glob(garmin_connect_masa_ciala)

dfs = []
for path in file_paths:
    df = pd.read_csv(path)
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)
df = df.iloc[:, :2]

In [57]:
data_values = df.values.tolist()
merged_data = [data_values[i] + data_values[i+1] for i in range(0, len(data_values)-1)]
df = pd.DataFrame(merged_data, columns=list(df.columns)*2)

index = df.columns.tolist().index('Czas', 1)
df.columns.values[index] = 'Czas2'
df = df[~df['Czas'].str.contains(':')]

df = df.iloc[:, [0, 2, 3]]

In [58]:
def replace_month(date_str):
    months_pl_to_en = {
        'Sty': 'Jan', 'Lut': 'Feb', 'Mar': 'Mar', 'Kwi': 'Apr',
        'Maj': 'May', 'Cze': 'Jun', 'Lip': 'Jul', 'Sie': 'Aug',
        'Wrz': 'Sep', 'Paź': 'Oct', 'Lis': 'Nov', 'Gru': 'Dec'
    }
    for pl, en in months_pl_to_en.items():
        date_str = date_str.replace(pl, en)
    return date_str.strip()

df_copy = df.copy()
df_copy.loc[:, 'Czas'] = df_copy['Czas'].apply(replace_month)
df_copy.loc[:, 'Czas'] = pd.to_datetime(df_copy['Czas'], format="%d %b %Y").dt.strftime('%Y-%m-%d')
df = df_copy

In [59]:
df = df.rename(columns={'Czas': 'Date', 'Czas2' : 'Time', 'Ciężar': 'Weight'})
df = df.reindex(columns=['Date', 'Time', 'Weight'])
df['Weight'] = df['Weight'].str.replace(' kg', '').astype(float)

In [60]:
duplicated_rows = df.duplicated()
print(f"Number of duplicated rows: {duplicated_rows.sum()}")

if duplicated_rows.sum() > 0:
  df = df.drop_duplicates()
  print (f"Found {duplicated_rows.sum()} rows. Deleted.")

Number of duplicated rows: 6
Found 6 rows. Deleted.


In [61]:
df = df.sort_values(by=['Date', 'Time'], ascending=True)
df_garmin_connect = df

In [62]:
print(f'Number of entries: {len(df_garmin_connect)}')
print(f'Date of first entry: {df_garmin_connect["Date"].iloc[0]}')
print(f'Date of last entry: {df_garmin_connect["Date"].iloc[-1]}')

Number of entries: 378
Date of first entry: 2020-12-06
Date of last entry: 2024-03-01


**Data Consalidation**

In [63]:
df_merged = pd.concat([df_google_sheets, df_garmin_connect], ignore_index=True)
duplicated_rows = df_merged.duplicated()

print(f"Number of duplicated rows: {duplicated_rows.sum()}")

if duplicated_rows.sum() > 0:
  df_merged = df_merged.drop_duplicates()
  print (f"Found {duplicated_rows.sum()} rows. Deleted")

Number of duplicated rows: 0


In [64]:
print(f'Number of entries: {len(df_merged)}')
print(f'Date of first entry: {df_merged["Date"].iloc[0]}')
print(f'Date of last entry: {df_merged["Date"].iloc[-1]}')

Number of entries: 428
Date of first entry: 2020-02-02
Date of last entry: 2024-03-01


In [65]:
# Export the merged dataframe to the weight.csv file
df_merged.to_csv('/content/drive/MyDrive/Colab Notebooks/Tableau/Outputs/weight.csv', index=False)